<a href="https://colab.research.google.com/github/deltorobarba/chemistry/blob/main/H2O_fermionic_hamiltonian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Water Molecule (H₂O) - Fermionic (Molecular) Hamiltonian with Second Quantization**

In [ ]:
!pip install pyscf matplotlib openfermion openfermionpyscf -q

In [ ]:
# Fermionic (Molecular) Hamiltonian with Second Quantization
from pyscf import gto, scf, ao2mo
from openfermion import MolecularData, transforms
from openfermion.transforms import jordan_wigner
from openfermionpyscf import generate_molecular_hamiltonian
from openfermion.transforms import bravyi_kitaev

# Define molecule
geometry = [
    ('O', (0.0000000, 0.0000000, 0.0000000)),
    ('H', (0.7586022, 0.0000000, 0.5042847)),
    ('H', (-0.7586022, 0.0000000, 0.5042847))
]

basis = 'cc-pvdz' # Smaller basis set (e.g., sto-3g instead of cc-pvdz) to reduce the number of orbitals involved.
charge = 0
multiplicity = 1

# Create a MolecularData object
molecule = MolecularData(geometry, basis, multiplicity, charge)

# Perform a Hartree-Fock calculation
mol = gto.M(
    atom=geometry,
    basis=basis,
    charge=charge,
    spin=multiplicity - 1
)

mf = scf.RHF(mol)
mf.kernel()

# Number of electrons and orbitals for the molecule
num_electrons = mol.nelectron
num_orbitals = mf.mo_coeff.shape[1]

# Manually extract integrals from PySCF
h1 = mf.get_hcore()  # One-electron integrals
eri = ao2mo.kernel(mol, mf.mo_coeff)  # Two-electron integrals
nuclear_repulsion = mol.energy_nuc()

# Use OpenFermion to generate the molecular Hamiltonian
molecular_hamiltonian = generate_molecular_hamiltonian(
    geometry=geometry,
    basis=basis,
    multiplicity=multiplicity,
    charge=charge,
    n_active_electrons=num_electrons,
    n_active_orbitals=num_orbitals
)

# Get the Hamiltonian in second-quantized form
fermion_hamiltonian = transforms.get_fermion_operator(molecular_hamiltonian)

# Convert the fermionic Hamiltonian to a qubit Hamiltonian using Jordan-Wigner transformation
qubit_hamiltonian = jordan_wigner(fermion_hamiltonian)
# qubit_hamiltonian = bravyi_kitaev(fermion_hamiltonian) # To speed up: Convert to qubit Hamiltonian using Bravyi-Kitaev transform

# Print the qubit Hamiltonian
print("Qubit Hamiltonian (Jordan-Wigner):\n", qubit_hamiltonian)

**This can be computationally intensive because:**
* Number of Orbitals and Electrons: If your molecule has many orbitals (e.g., a large basis set) or electrons, the Hamiltonian will involve many terms. The Jordan-Wigner transformation processes each of these terms and converts them to qubit operators, which can take time.
* Two-Electron Integrals: These integrals contribute many terms to the Hamiltonian, and the mapping from fermions to qubits adds further complexity.

**To speed this up**:
* Try using a **smaller basis set** (e.g., sto-3g instead of cc-pvdz) to reduce the number of orbitals involved.
* **Active Space Approximation**: You can reduce the number of orbitals by focusing on an "active space" that includes only the most chemically relevant orbitals. For example, in OpenFermion-PySCF, you can define an active space of electrons and orbitals to restrict the number of orbitals used in the calculation.
* Use the **Bravyi-Kitaev transformation**: The Bravyi-Kitaev transformation may sometimes be faster and more efficient than the Jordan-Wigner transformation for certain systems, as it has better scaling for specific problem sizes.

**To speed up: Active Space Approximation and Smaller Basis Set**
* n_active_electrons = 4: Specifies the number of active electrons in the calculation. You can adjust this value based on your molecule's complexity and the level of approximation you're willing to make.
* n_active_orbitals = 6: Specifies the number of active orbitals, reducing the size of the problem.

In [ ]:
from pyscf import gto, scf, ao2mo
from openfermion import MolecularData, transforms
from openfermion.transforms import jordan_wigner
from openfermionpyscf import generate_molecular_hamiltonian

# Define the molecule
geometry = [
    ('O', (0.0000000, 0.0000000, 0.0000000)),
    ('H', (0.7586022, 0.0000000, 0.5042847)),
    ('H', (-0.7586022, 0.0000000, 0.5042847))
]
basis = 'sto-3g'  # Smaller basis set
charge = 0
multiplicity = 1

# Perform a Hartree-Fock calculation
mol = gto.M(
    atom=geometry,
    basis=basis,
    charge=charge,
    spin=multiplicity - 1
)

mf = scf.RHF(mol)
mf.kernel()

# Print the number of molecular orbitals and electrons
num_orbitals = mf.mo_coeff.shape[1]
num_electrons = mol.nelectron

print(f"Total number of orbitals: {num_orbitals}")
print(f"Total number of electrons: {num_electrons}")

# Set the active space (ensure these values are within the total available)
n_active_electrons = min(4, num_electrons)  # Adjust to be <= total number of electrons
n_active_orbitals = min(4, num_orbitals)    # Adjust to be <= total number of orbitals

# Use OpenFermion-PySCF to generate the molecular Hamiltonian with the active space
molecular_hamiltonian = generate_molecular_hamiltonian(
    geometry=geometry,
    basis=basis,
    multiplicity=multiplicity,
    charge=charge,
    n_active_electrons=n_active_electrons,
    n_active_orbitals=n_active_orbitals
)

# Get the Hamiltonian in second-quantized form
fermion_hamiltonian = transforms.get_fermion_operator(molecular_hamiltonian)

# Convert the fermionic Hamiltonian to a qubit Hamiltonian using Jordan-Wigner transformation
qubit_hamiltonian = jordan_wigner(fermion_hamiltonian)

# Print the qubit Hamiltonian
print("Qubit Hamiltonian (Jordan-Wigner):\n", qubit_hamiltonian)